In [1]:
pip install scx>=1.2.1 #Note: This may throw an Error in Google Colab since Colab uses an outdated duckdb package. This file should still work in Colab without issues.

Note: you may need to restart the kernel to use updated packages.


<h1>Blinky P1</h1>
<p>After years of work, your friend Robert, and you launched the brand Blink. Blink's business includes assembling, selling, and distributing a smart pet wearable device, the Blinky22. This device has multiple functions; it monitors pet activity levels, tracks health indicators, provides wellness recommendations, records veterinarian visits, tracks location, and shares this all in real-time through a mobile app. Blinky22 is waterproof and light-weight. The collar is adjustable so that any pet can wear it.</p>
<p>Blinky is manufactured in two assembly plants and it is sold in three regions. Monthly demand per region is shared in Table 1. Currently, assembly plants have no capacity restrictions and can source as many items as needed. Blink’s 3PL carrier charges a transportation cost of (USD)0.12 per unit per mile.</p>
<p><b>Table 1: Demand in units</b></p>
<table width="90%">
<tbody>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black;">Demand</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Region 1</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Region 2</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Region 3</td>
</tr>
<tr>
<td width="15%" style="text-align: center; border: 1px solid black;">Units per month</td>
<td width="10%" style="text-align: center; border: 1px solid black;">2500</td>
<td width="10%" style="text-align: center; border: 1px solid black;">4350</td>
<td width="10%" style="text-align: center; border: 1px solid black;">3296</td>
</tr>
</tbody>
</table>
<p><b>Table 2: Distance in Miles</b></p>
<table width="90%" style="height: 76.7814px;">
<tbody>
<tr style="height: 25.5938px;">
<td width="15%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Miles</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Region 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Region 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Region 3</td>
</tr>
<tr style="height: 25.5938px;">
<td width="15%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Assembly Plant 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">105</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">256</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">86</td>
</tr>
<tr style="height: 25.5938px;">
<td width="15%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Assembly Plant 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">240</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">136</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">198</td>
</tr>
</tbody>
</table>
<p></p>
<p><strong>Formulate a model using the available information. Your goal is to minimize the total transportation cost.</strong></p>

In [2]:
from scx.optimize import Model

In [3]:
distances=[
    [105, 256, 86],
    [240, 136, 198],
]
cost_per_mi=0.12

demand=[2500,4350,3296]

assembly_plants=['A1','A2']
demand_regions=['R1','R2','R3']

decision_variables=[[Model.variable(name=f"{a}__{d}__amt", lowBound=0) for d in demand_regions] for a in assembly_plants]


In [4]:
# Initialize the model
my_model = Model(name="Blinky22", sense='minimize')

# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([distances[i][j]*decision_variables[i][j]*cost_per_mi for i in range(len(assembly_plants)) for j in range(len(demand_regions))]),
)

# Add the Constraints
## Demand Constraint
for j in range(len(demand_regions)):
    my_model.add_constraint(
        name=f"Constraint__{demand_regions[j]}",
        fn=Model.sum([decision_variables[i][j] for i in range(len(assembly_plants))])>=demand[j]
    )

my_model.solve()

In [5]:
# Show the formulation
my_model.show_formulation()

Blinky22:
MINIMIZE
12.6*A1__R1__amt + 30.72*A1__R2__amt + 10.32*A1__R3__amt + 28.799999999999997*A2__R1__amt + 16.32*A2__R2__amt + 23.759999999999998*A2__R3__amt + 0.0
SUBJECT TO
Constraint__R1: A1__R1__amt + A2__R1__amt >= 2500

Constraint__R2: A1__R2__amt + A2__R2__amt >= 4350

Constraint__R3: A1__R3__amt + A2__R3__amt >= 3296

VARIABLES
A1__R1__amt Continuous
A1__R2__amt Continuous
A1__R3__amt Continuous
A2__R1__amt Continuous
A2__R2__amt Continuous
A2__R3__amt Continuous



In [6]:
# Show the outputs
my_model.show_outputs()

{'objective': 136506.72,
 'status': 'Optimal',
 'variables': {'A1__R1__amt': 2500.0,
               'A1__R2__amt': 0.0,
               'A1__R3__amt': 3296.0,
               'A2__R1__amt': 0.0,
               'A2__R2__amt': 4350.0,
               'A2__R3__amt': 0.0}}
